In [ ]:
import torch
from torch import nn
import torchvision
import torch.nn.functional as F
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import seaborn as sns

In [ ]:
from layers import BatchGraphAttentionLayer

class GATAuto(nn.Module):
    def __init__(self, in_features, n_nodes, n_classes, hidden_sizes, dropout_p):
        super(GATAuto, self).__init__()

        self.dropout_p = dropout_p
        self.gat = BatchGraphAttentionLayer(in_features=in_features, out_features=hidden_sizes[0], dropout=dropout_p, alpha=0.2)

        self.flatten = nn.Flatten()
        self.linear = nn.Linear(hidden_sizes[0] * n_nodes, n_classes)

        self.node_embeddings = nn.Parameter(torch.FloatTensor(n_nodes, n_nodes))

    def forward(self, x):
        out = F.relu(self.gat(x, self.node_embeddings))

        out = self.flatten(out)
        out = self.linear(out)

        return out

    def init_node_embeddings(self):
        stdv = 1. / math.sqrt(self.node_embeddings.size(1))
        self.node_embeddings.data.uniform_(-stdv, stdv)
        self.node_embeddings.data.fill_diagonal_(1)

model = GATAuto(in_features=28, n_nodes=28, n_classes=10, hidden_sizes=[64], dropout_p=0)
model.init_node_embeddings()
model.train()
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, 1.0)

In [ ]:
class Sqeeze(object):
    def __call__(self, sample):
        sqzd = torch.squeeze(sample)
        return sqzd
    
    def __repr__(self) -> str:
        return 'Squeze()'

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081)),
    Sqeeze()
])

mnist_train = torchvision.datasets.MNIST(root='./dataset', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.MNIST(root='./dataset', train=False, download=True, transform=transform)

train_dataloader = torch.utils.data.DataLoader(mnist_train, batch_size=32, shuffle=True, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(mnist_test, batch_size=32, shuffle=True, drop_last=True)

print(mnist_train)

In [ ]:
for input, label in train_dataloader:
    input = input.to(device)
    label = label.to(device)
    print(input.shape)
    print(label.shape)
    break

# Adj before

In [ ]:
adj_before = model.node_embeddings.cpu().detach()


In [ ]:
from train import train_model_2

dataset_sizes = {'train': len(train_dataloader), 'val': len(test_dataloader), 'test': len(test_dataloader)}
dataloaders = {'train': train_dataloader, 'val': test_dataloader, 'test': test_dataloader}
class_names = None
model, history = train_model_2(model, optimizer, criterion, train_dataloader, test_dataloader, epochs=2, random_seed=0, device=device)

# Adj after

In [ ]:
adj_after = model.node_embeddings.cpu().detach()


In [ ]:
import numpy as np
np.unique(adj_before == adj_after)

In [ ]:
history['loss']

# Pytorch Geometric

In [1]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

OSError: /home/amar/Desktop/thesis-2022/env/lib/python3.7/site-packages/torch_sparse/_convert_cuda.so: undefined symbol: _ZNK2at6Tensor5zero_Ev